- De antrenat un face detector si de testat pe video-uri
- https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html
- https://www.kaggle.com/jessicali9530/celeba-dataset/download
- https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

Smile detector   
Image Classification -> Supervised->Classification -> Binary Classification       
Folosim retea preantrenata si eliminam ultimele layere liniare, Data generator/ Data loader

###Imports

In [3]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
import torch.optim as optim
import torch.nn as nn
from PIL import Image
from torchvision.models import regnet_y_400mf

In [4]:
if torch.cuda.is_available(): 
  dev = "cuda:0" 
else: 
  dev = "cpu" 
device = torch.device(dev)

In [5]:
dev

'cuda:0'

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# import os

# os.path.isfile("/content/drive/MyDrive/Colab\ Notebooks/CelebA_Project/archive.zip")
# !unzip /content/drive/MyDrive/Colab\ Notebooks/CelebA_Project/archive.zip -d /content/drive/MyDrive/Colab\ Notebooks/CelebA_Project/

In [ ]:
# img_dir = "/content/drive/MyDrive/Colab Notebooks/CelebA_Project/img_align_celeba/img_align_celeba"
# import os
# for ix, file in enumerate(os.listdir(img_dir)):
#     if ix > 5000:
#         os.remove(os.path.join(img_dir, file))

In [6]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CelebA_Project/archive/list_attr_celeba.csv")

In [7]:
model = regnet_y_400mf(weights='IMAGENET1K_V1', )

In [8]:
print(model)

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=6, bias=False)
            

In [9]:
print(model.fc)

Linear(in_features=440, out_features=1000, bias=True)


In [10]:
model.fc = torch.nn.Linear(in_features=440, out_features=2, bias=True)

In [11]:
print(model.fc)

Linear(in_features=440, out_features=2, bias=True)


In [24]:
model.to(device)

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=6, bias=False)
            

In [25]:
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((128, 128)),
                                            torchvision.transforms.ToTensor()])

In [26]:
class CelebADataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, validation=False):
        self.img_labels = pd.read_csv(annotations_file)
        valid_images = os.listdir(img_dir) # imi iau imaginile din directorul din drive
        if validation == True:
            valid_images = valid_images[int(0.8*len(valid_images)):]
        else:
            valid_images = valid_images[:int(0.8*len(valid_images))]
        self.img_labels = self.img_labels[self.img_labels["image_id"].isin(valid_images)]# ca sa fie aceleasi imagini pe drive si in csv
        self.img_labels = self.img_labels.reset_index()
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.loc[idx, "image_id"]) # ia din image_id fiecare index pentru imagine
        image = Image.open(img_path)
        label = self.img_labels.loc[idx, "Smiling"]
        if label == -1:
            label = label + 1  # ca sa avem etichetele 0 si 1
            # label = 0
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

CelebA_dataset_train = CelebADataset(annotations_file="/content/drive/MyDrive/Colab Notebooks/CelebA_Project/archive/list_attr_celeba.csv",
                               img_dir="/content/drive/MyDrive/Colab Notebooks/CelebA_Project/archive/img_align_celeba/img_align_celeba", transform=transform, validation=False)
CelebA_dataset_validation = CelebADataset(annotations_file="/content/drive/MyDrive/Colab Notebooks/CelebA_Project/archive/list_attr_celeba.csv",
                               img_dir="/content/drive/MyDrive/Colab Notebooks/CelebA_Project/archive/img_align_celeba/img_align_celeba", transform=transform, validation=True)

In [27]:
from torch.utils.data import DataLoader

trainloader = DataLoader(CelebA_dataset_train, batch_size=64, shuffle=True, num_workers=4)
validloader = DataLoader(CelebA_dataset_validation, batch_size=64, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [28]:
print(CelebA_dataset_train.__len__())

8000


In [29]:
print(CelebA_dataset_train.__getitem__(7000))

(tensor([[[0.8902, 0.8902, 0.8902,  ..., 0.8824, 0.8784, 0.8784],
         [0.8902, 0.8902, 0.8902,  ..., 0.8941, 0.8784, 0.8784],
         [0.8902, 0.8902, 0.8902,  ..., 0.8824, 0.8745, 0.8745],
         ...,
         [0.2000, 0.2000, 0.2000,  ..., 0.2431, 0.2431, 0.2431],
         [0.2000, 0.2000, 0.2000,  ..., 0.2431, 0.2471, 0.2471],
         [0.2000, 0.2000, 0.2000,  ..., 0.2431, 0.2471, 0.2471]],

        [[0.8902, 0.8902, 0.8902,  ..., 0.8941, 0.8824, 0.8824],
         [0.8902, 0.8902, 0.8902,  ..., 0.9059, 0.8824, 0.8824],
         [0.8902, 0.8902, 0.8902,  ..., 0.8941, 0.8784, 0.8784],
         ...,
         [0.1961, 0.1961, 0.1961,  ..., 0.2392, 0.2392, 0.2392],
         [0.1961, 0.1961, 0.1961,  ..., 0.2392, 0.2431, 0.2431],
         [0.1961, 0.1961, 0.1961,  ..., 0.2392, 0.2431, 0.2431]],

        [[0.8824, 0.8824, 0.8824,  ..., 0.8667, 0.8588, 0.8588],
         [0.8824, 0.8824, 0.8824,  ..., 0.8784, 0.8588, 0.8588],
         [0.8824, 0.8824, 0.8824,  ..., 0.8667, 0.8549, 0

In [30]:
trainloader

Inlocuirea ultimului layer din modelul importat

In [31]:
# presupunem ca am creat deja dataset-ul/dataloader-ul

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) #(nesterov momentum)

In [35]:
# Train the network

from time import time
from tqdm import tqdm 

num_epochs = 20

checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/CelebA_Project/checkpoint_experiment1"
if os.path.isdir(checkpoint_dir) == False:
    os.makedirs(checkpoint_dir)

steps_to_checkpoint = 50

for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()

    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()        
        optimizer.step()
        #end = time()
        #print("We spent ", end - start, "on fwd+bwd the data")
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0
        if i % steps_to_checkpoint == 49:
            torch.save(model.state_dict(), os.path.join(checkpoint_dir, "model_step_" + str(i) + "_epoch_" + str(epoch) + ".pth"))
        
    model.eval() # opresc orice actualizare de parametri
    running_loss = 0
    corrects = 0
    for j, data in tqdm(enumerate(validloader, 0)):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
          outputs = model(inputs)
          prob_outputs = nn.Softmax(dim=-1)(outputs)
        predicted_labels = torch.argmax(prob_outputs, dim=1)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        corrects += torch.sum(predicted_labels == labels)
    print(f'[{epoch + 1}] loss: {running_loss / (j+1):.3f}')
    print("Accuracy is ", corrects / ((j+1)*64))

print('Finished Training')

125it [00:37,  3.36it/s]
32it [00:06,  5.26it/s]

[1] loss: 0.497
Accuracy is  tensor(0.7412, device='cuda:0')



125it [00:36,  3.41it/s]
32it [00:06,  5.25it/s]

[2] loss: 0.499
Accuracy is  tensor(0.7432, device='cuda:0')



125it [00:36,  3.38it/s]
32it [00:06,  5.22it/s]

...

[9] loss: 0.501
Accuracy is  tensor(0.7422, device='cuda:0')



125it [00:37,  3.36it/s]
32it [00:06,  5.32it/s]

[10] loss: 0.500
Accuracy is  tensor(0.7432, device='cuda:0')



52it [00:15,  3.27it/s]


KeyboardInterrupt: ignored

Inferenta

In [40]:
checkpoint_model = "/content/drive/MyDrive/Colab Notebooks/CelebA_Project/checkpoint_experiment1/model_step_49_epoch_5.pth"

model = regnet_y_400mf(weights='IMAGENET1K_V1', )
model.fc = torch.nn.Linear(in_features=440, out_features=2, bias=True)
model.load_state_dict(torch.load(checkpoint_model))


<All keys matched successfully>

In [41]:
model.eval()
model.to(device)
running_loss = 0
corrects = 0
    
for j, data in tqdm(enumerate(validloader, 0)):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(inputs)
      prob_outputs = nn.Softmax(dim=-1)(outputs)
    predicted_labels = torch.argmax(prob_outputs, dim=1)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
    corrects += torch.sum(predicted_labels == labels)
print(f'[{epoch + 1}] loss: {running_loss / (j+1):.3f}')
print("Accuracy is ", corrects / ((j+1)*64))

print('Finished Training')

32it [00:06,  4.79it/s]

[11] loss: 0.502
Accuracy is  tensor(0.7441, device='cuda:0')
Finished Training
